In [1]:
pip install dcor

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tigramite

Note: you may need to restart the kernel to use updated packages.


In [3]:
! mamba install -y pynco pynio pyngl


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.1.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pynco', 'pynio', 'pyngl']

conda-forge/linux-64                                        Using cache

In [4]:
import gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cftime
from nco import Nco
from tqdm import tqdm
import Ngl #plotting science data
import xesmf as xe

import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import parcorr, gpdc, cmiknn

from utils.constants import TAU_MIN, TAU_MAX
import utils.links as links
import utils.helpers as helpers

In [5]:
optional_ozone = 'pbuf_ozone'
predictor_names = ['pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX', 'state_ps', 'state_q0001', 'state_t']
predictand_names = ['cam_out_PRECC']
ds = helpers.load_vars_xarray(predictor_names, predictand_names).isel(lev=30, ncol=0)
ds

<xarray.Dataset>
Dimensions:            (time: 2920)
Coordinates:
  * time               (time) object 0001-02-01 12:00:00 ... 0009-01-31 12:00:00
    ncol               int64 0
Data variables:
    pbuf_SOLIN         (time) float64 ...
    pbuf_LHFLX         (time) float64 ...
    pbuf_SHFLX         (time) float64 ...
    state_ps           (time) float64 ...
    state_q0001        (time) float64 ...
    state_t            (time) float64 ...
    out_cam_out_PRECC  (time) float64 ...
Attributes:
    calendar:  NO_LEAP
    fv_nphys:  2
    ne:        4

In [6]:
# fs = gcsfs.GCSFileSystem()
# fs.ls("gs://leap-persistent-ro/sungdukyu")

# mapper_in = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.input.zarr')
# mapper_out = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.output.zarr')
# mapper_grid = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.grid-info.zarr')

# ds_train_in = xr.open_dataset(mapper_in, engine='zarr')
# ds_train_out = xr.open_dataset(mapper_out, engine='zarr')
# gridpoints = xr.open_dataset(mapper_grid, engine ='zarr')

## Reformat the Data in Time of Day

In [7]:
# # see above: **ymd** includes date information: the first digit indicates the index of year, the next two digits indicate the month and the last three digits indicates the calendar day in the year.
# year=ds_train_in['ymd']//10000 # gets first digit
# month=ds_train_in['ymd']%10000//100 # gets 4 right digits then gets the 2 left digits
# day=ds_train_in['ymd']%10000%100 # gets 4 right digits then gets the 2 right digits
# hour=ds_train_in['tod']//3600 # seconds per minute, minutes per hour
# minute=ds_train_in['tod']%3600//60 # removes hours, leaves number of seconds in the day, then divide by 60 to get minutes

# t = []
# for k in range(len(ds_train_in['ymd'])): # go through ds
#     t.append(cftime.DatetimeNoLeap(year[k],month[k],day[k],hour[k],minute[k]))
# ds_train_in['sample'] = t # Add the time array to the 'sample' dimension and rename
# ds_train_in = ds_train_in.rename({'sample':'time'})
# ds_train_in = ds_train_in.drop(['tod','ymd'])

# ds_train_in.time.values[0:5] # Check the current **time** dimension, read the timestep

In [8]:
# # see above: **ymd** includes date information: the first digit indicates the index of year, the next two digits indicate the month and the last three digits indicates the calendar day in the year.
# year=ds_train_out['ymd']//10000 # gets first digit
# month=ds_train_out['ymd']%10000//100 # gets 4 right digits then gets the 2 left digits
# day=ds_train_out['ymd']%10000%100 # gets 4 right digits then gets the 2 right digits
# hour=ds_train_out['tod']//3600 # seconds per minute, minutes per hour
# minute=ds_train_out['tod']%3600//60 # removes hours, leaves number of seconds in the day, then divide by 60 to get minutes

# t = []
# for k in range(len(ds_train_out['ymd'])): # go through ds
#     t.append(cftime.DatetimeNoLeap(year[k],month[k],day[k],hour[k],minute[k]))
# ds_train_out['sample'] = t # Add the time array to the 'sample' dimension and rename
# ds_train_out =ds_train_out.rename({'sample':'time'})
# ds_train_out = ds_train_out.drop(['tod','ymd'])

# ds_train_out.time.values[0:5] # Check the current **time** dimension, read the timestep

In [9]:
# ds_train_in.time.values[0:5]

In [10]:
# optional_ozone = 'pbuf_ozone'
# predictor_names = ['pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX', 'state_ps', 'state_q0001', 'state_t']
# predictand_names = ['cam_out_PRECC']

# ds_predictors = ds_train_in.isel(lev=30, ncol=0)[predictor_names]
# ds_predictands = ds_train_out.isel(lev=30, ncol=0)[predictand_names]
# ds_predictors


In [11]:
# # Suppose 'da1' and 'da2' are your xarray DataArrays
# combined = xr.merge([ds_predictors, ds_predictands])

In [17]:
# # Convert the Dataset to a DataArray
da = ds.to_array(dim='variable')
# da

In [13]:
# da.time

In [14]:
# da.T.shape

In [19]:
# dataframe = pp.DataFrame(da.T.values,
#                          datatime=da.time.values,
#                          var_names=predictor_names.extend(predictand_names))

# ParCorr = parcorr.ParCorr(significance='analytic')
# pcmci = PCMCI(dataframe=dataframe, cond_ind_test=ParCorr, verbosity=1)

var_names = predictor_names.extend(predictand_names)
dataframe = pp.DataFrame(da.T.values,
                         datatime=da.time.values,
                         var_names=var_names)

ParCorr = parcorr.ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=ParCorr, verbosity=1)

In [ ]:
pcmci.verbosity = 1
tau_max=2
results = pcmci.run_pcmci(tau_max=tau_max, pc_alpha=None)
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=tau_max, fdr_method='fdr_bh')

tp.plot_graph(graph = results['graph'],
    val_matrix=results['val_matrix'],
    var_names=var_names,
    link_colorbar_label='MCI',
    node_colorbar_label='auto-MCI',
    link_label_fontsize=20,
    label_fontsize=20,
    tick_label_size=14,
    node_label_size=20,
    edge_ticks=0.5,
    node_ticks=0.5,
    node_size=0.5
    )


##
## Step 1: PC1 algorithm with lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 2
pc_alpha = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
max_conds_dim = None
max_combinations = 1




In [ ]:
tp.plot_time_series_graph(
    figsize=(4, 4),
    val_matrix=results['val_matrix'],
    graph = results['graph'],
    var_names=var_names,
    link_colorbar_label='MCI',
    label_fontsize=20,
    tick_label_size=20
)

In [ ]:
pip install dcor